In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os
import numpy as np

In [2]:
# Load the dataset
paintings = pd.read_csv('paintings.csv', dtype={'CreationYear': int, 'Height': float, 'Medium': str, 'Price': float})

paintings.head()

,CreationYear,Height,Medium,Price
0,1826,0.86,Pastel,9131.21
1,1982,0.78,Pastel,8182.09
2,1943,0.98,Pastel,1466.33
3,1826,2.60,Oil,5717.82
4,1835,2.68,Digital,7173.12


In [3]:
# One-hot encoding for  column
data = pd.get_dummies(paintings, columns=['Medium'])

In [4]:
# Split the data
X = data.drop(columns=['Price'])
y = data['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Preprocess the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Load data
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [6]:
# Define the model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 64),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
)

In [7]:
# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [8]:
# Train the model
n_epochs = 1000
min_val_loss = np.inf
for epoch in range(n_epochs):
    for inputs, targets in train_loader:
        # Forward pass
        output = model(inputs)
        loss = criterion(output, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss for every epoch
        print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

    # Early stopping
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for inputs, targets in test_loader:
            output = model(inputs)
            val_loss += criterion(output, targets).item()
        if val_loss < min_val_loss:
            min_val_loss = val_loss
        else:
            print(f'Early stopping on epoch {epoch}')
            break

    model.train()

Epoch: 1, Loss: 40149824.0
Epoch: 1, Loss: 29790892.0
Epoch: 1, Loss: 37752772.0
Epoch: 1, Loss: 33727820.0
Epoch: 1, Loss: 32466446.0
Epoch: 1, Loss: 37898212.0
Epoch: 1, Loss: 39916488.0
Epoch: 1, Loss: 36101920.0
Epoch: 1, Loss: 49738112.0
Epoch: 1, Loss: 33860552.0
Epoch: 1, Loss: 36687016.0
Epoch: 1, Loss: 34015092.0
Epoch: 1, Loss: 37755160.0
Epoch: 1, Loss: 34913156.0
Epoch: 1, Loss: 35722044.0
Epoch: 1, Loss: 42152352.0
Epoch: 1, Loss: 34066788.0
Epoch: 1, Loss: 32774970.0
Epoch: 1, Loss: 44252552.0
Epoch: 1, Loss: 33102606.0
Epoch: 1, Loss: 35269880.0
Epoch: 1, Loss: 32796264.0
Epoch: 1, Loss: 37983248.0
Epoch: 1, Loss: 35829692.0
Epoch: 1, Loss: 34915168.0
Epoch: 1, Loss: 32371132.0
Epoch: 1, Loss: 37004516.0
Epoch: 1, Loss: 31582768.0
Epoch: 1, Loss: 45013932.0
Epoch: 1, Loss: 29562718.0
Epoch: 1, Loss: 35321996.0
Epoch: 1, Loss: 32803402.0
Epoch: 1, Loss: 32881000.0
Epoch: 1, Loss: 30112730.0
Epoch: 1, Loss: 37609536.0
Epoch: 1, Loss: 40777076.0
Epoch: 1, Loss: 32470886.0
E

In [9]:
# Evaluate the model
model.eval()
with torch.no_grad():
    total_loss = 0
    total_count = 0
    for inputs, targets in test_loader:
        output = model(inputs)
        loss = criterion(output, targets)
        total_loss += loss.item() * inputs.size(0)
        total_count += inputs.size(0)
    mse = total_loss / total_count
    print('MSE:', mse)

MSE: 6766935.25904


In [10]:
# Save the model
if not os.path.exists("Models"):
    os.makedirs("Models")

model_path = 'Models/paintingModel.pt'

# Save the model
torch.save(model.state_dict(), model_path)

In [12]:
# Save the model in ONNX format

# Dummy input values
creation_year = 1900
height = 2.0
medium = "Oil"

# Convert the medium to a one-hot encoded vector
medium_dict = {"Oil": 0, "Acrylic": 1, "Watercolor": 2, "Pastel": 3, "Charcoal": 4, "Digital": 5}
medium_one_hot = np.zeros(len(medium_dict), dtype=np.float32)
medium_one_hot[medium_dict[medium]] = 1.0

# Create the dummy input tensor
dummy_input = torch.tensor([[creation_year, height] + list(medium_one_hot)], dtype=torch.float32)

# Create an instance of your model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 64),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
)

# Load the model weights
model.load_state_dict(torch.load('Models/paintingModel.pt'))

# Set the model to evaluation mode
model.eval()

# Export the model to ONNX format using dummy_input
torch.onnx.export(model, dummy_input, 'paintingModel.onnx', verbose=True)